# Heart Disease Detection Model Proposal

## Introduction:

## Preliminary exploratory data analysis:

## Method: 

## Expected Results:

## References:

1. https://www.canada.ca/en/public-health/services/publications/diseases-conditions/heart-disease-canada.html 
2. https://archive.ics.uci.edu/dataset/45/heart+disease 


In [ ]:
!pip install pandas==1.5.3
!pip install altair==4.2.2
!pip install ucimlrepo==0.0.3

In [15]:
import pandas as pd
import altair as alt
import ucimlrepo

In [16]:
heart_disease = ucimlrepo.fetch_ucirepo(id=45) 

In [18]:
predictors = heart_disease.data.features 
target = heart_disease.data.targets 
cleveland = pd.concat([predictors, target], axis=1)
cleveland

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45,1,1,110,264,0,0,132,0,1.2,2,0.0,7.0,1
299,68,1,4,144,193,1,0,141,0,3.4,2,2.0,7.0,2
300,57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0,3
301,57,0,2,130,236,0,2,174,0,0.0,2,1.0,3.0,1


In [19]:
trestbps_chol_plot = alt.Chart(cleveland).mark_point().encode(
    x = alt.X("trestbps", title = "resting blood pressure"),
    y = alt.Y('chol', title = "serum cholestoral"),
    color = alt.Color("num", title = "Diagnosis of Heart diease, 0 means None")
)
trestbps_chol_plot

alt.Chart(...)

In [20]:
trestbps_thalach_plot = alt.Chart(cleveland).mark_point().encode(
    x = alt.X("trestbps", title = "resting blood pressure" ),
    y = alt.Y('thalach', title = "resting blood pressure"),
    color = alt.Color("num", title = "Diagnosis of Heart diease, 0 means None")
)
trestbps_thalach_plot

alt.Chart(...)

In [21]:
trestbps_oldpeak_plot = alt.Chart(cleveland).mark_point().encode(
    x = alt.X("trestbps", title = "resting blood pressure"),
    y = alt.Y('oldpeak', title = "ST depression induced by exercise"),
    color = alt.Color("num", title = "Diagnosis of Heart diease, 0 means None")
)
trestbps_oldpeak_plot

alt.Chart(...)

In [22]:
trestbps_ca_plot = alt.Chart(cleveland).mark_point().encode(
    x = alt.X("trestbps", title = "resting blood pressure"),
    y = alt.Y('ca', title = "Major vessels (0-3) coloured by fluoroscopy"),
    color = alt.Color("num", title = "Diagnosis of Heart diease, 0 means None")
)
trestbps_ca_plot

alt.Chart(...)

In [23]:
chol_thalach_plot = alt.Chart(cleveland).mark_point().encode(
    x = alt.X("chol", title = "serum cholestoral"),
    y = alt.Y('thalach', title = "resting blood pressure"),
    color = alt.Color("num", title = "Diagnosis of Heart diease, 0 means None")
)
chol_thalach_plot

alt.Chart(...)

In [24]:
chol_oldpeak_plot = alt.Chart(cleveland).mark_point().encode(
    x = alt.X("chol", title = "serum cholestoral"),
    y = alt.Y('oldpeak', title = "ST depression induced by exercise"),
    color = alt.Color("num", title = "Diagnosis of Heart diease, 0 means No heart disease")
)
chol_oldpeak_plot

alt.Chart(...)

In [25]:
chol_ca_plot = alt.Chart(cleveland).mark_point().encode(
    x = alt.X("chol", title = "serum cholestoral"),
    y = alt.Y('ca', title = "Major vessels (0-3) coloured by fluoroscopy"),
    color = alt.Color("num", title = "Diagnosis of Heart diease, 0 means No heart disease")
)
chol_ca_plot

alt.Chart(...)

In [26]:
oldpeak_thalach_plot = alt.Chart(cleveland).mark_point().encode(
    x = alt.X("oldpeak", title = "ST depression induced by exercise"),
    y = alt.Y('thalach', title = "resting blood pressure"),
    color = alt.Color("num", title = "Diagnosis of Heart diease, 0 means No heart disease")
)
oldpeak_thalach_plot

alt.Chart(...)

In [27]:
oldpeak_ca_plot = alt.Chart(cleveland).mark_point().encode(
    x = alt.X("oldpeak", title = "ST depression induced by exercise"),
    y = alt.Y('ca', title = "Major vessels (0-3) coloured by fluoroscopy"),
    color = alt.Color("num", title = "Diagnosis of Heart diease, 0 means No heart disease")
)
oldpeak_ca_plot

alt.Chart(...)